In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
#from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D , merge, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.core import RepeatVector, Permute
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader_ab import DataLoader
from skimage.io import imsave
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
import numpy as np
import os

Using TensorFlow backend.


In [2]:
class C1GAN():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.G_channels = 1
        self.D_channels = 2
        self.G_img_shape = (self.img_rows, self.img_cols, self.G_channels)
        self.D_img_shape = (self.img_rows, self.img_cols, self.D_channels)
        # Configure data loader
        self.dataset_name = 'place365'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
        
        
        
        optimizer = Adam(lr=0.0002, beta_1=0.5)
        optimizerD = Adam(lr=3.16e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizerD,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.D_img_shape) 
        img_B = Input(shape=self.G_img_shape)
        embeddings = Input(shape=(1000,))
        # By conditioning on B, throught inception embeddings, generate a fake version of A
        fake_A = self.generator([img_B , embeddings])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B , embeddings], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mse'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

    def build_generator(self):

        
        embed_input = Input(shape=(1000,))
        #Encoder
        encoder_input = Input(shape=(256, 256, 1,))
        encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
        #encoder_output = BatchNormalization(momentum=0.8)(encoder_output)
        encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
        encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
        encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
        encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
        encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
        encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
        encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

        #Fusion
        fusion_output = RepeatVector(32 * 32)(embed_input) 
        fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
        fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
        fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

        #Decoder
        decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
        decoder_output = UpSampling2D((2, 2))(decoder_output)
        decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
        decoder_output = UpSampling2D((2, 2))(decoder_output)
        decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
        decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
        decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
        decoder_output = UpSampling2D((2, 2))(decoder_output)

        model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

        return model

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=self.D_img_shape)
        img_B = Input(shape=self.G_img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([img_A, img_B], validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            
            for batch_i, (imgs_A, embeddings, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                #imgs_B = np.expand_dims(imgs_B, axis=0)
                fake_A = self.generator.predict([imgs_B , embeddings])

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B , embeddings], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
               
            self.save(epoch)
            print("weights saved...")
            
            

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/C1-%s-%d' % (epoch , batch_i), exist_ok=True)
        r, c = 3, 3

        imgs_A, embed ,imgs_B = self.data_loader.load_data(batch_size=40)
        fake_A = self.generator.predict([imgs_B , embed])
        fake_A = fake_A * 128

        # Output colorizations
        for i in range(len(fake_A)):
            cur = np.zeros((256, 256, 3))
            cur[:,:,0] = imgs_B[i][:,:,0]
            cur[:,:,1:] = fake_A[i]
            imsave("images/C1-%s-%d/gen_%d.png" % (epoch,batch_i, i), lab2rgb(cur))
            
    def sample_imagesOLD(self, epoch, batch_i , save_model = True):    
        os.makedirs('images/C1-%s-%d' % (epoch , batch_i), exist_ok=True)
        imgs_A, embed ,imgs_B = self.data_loader.load_data(batch_size=25)
        fake_A = self.generator.predict([imgs_B , embed])
        fake_A = fake_A * 128
        imgs_A = imgs_A*128
        
        titles = ['B&W', 'Generated', 'Original']
        
        
        for i in range(len(fake_A)):

            fig, axs = plt.subplots(3)
            axs[0].set_title(titles[0])
            axs[1].set_title(titles[1])
            axs[2].set_title(titles[2])
            
            gen = np.zeros((256, 256, 3))
            gen[:,:,0] = imgs_B[i][:,:,0]
            gen[:,:,1:] = fake_A[i]
            rgb_gen = lab2rgb(gen)
            
            real = np.zeros((256, 256, 3))
            real[:,:,0] = imgs_B[i][:,:,0]
            real[:,:,1:] = imgs_A[i]
            rgb_real = lab2rgb(real)
            
            
            gen = np.zeros((256, 256, 3))
            gen[:,:,0] = imgs_B[i][:,:,0]
            gen[:,:,1:] = imgs_A[i]/128
            rgb_bew = lab2rgb(gen)

            
            
            axs[0].imshow(rgb_bew)
            axs[1].imshow(rgb_gen)
            axs[2].imshow(rgb_real)
            
            #imsave("images/C1-%s-%d/%d_bew.png" % (epoch,batch_i, i), rgb_bew)
            #imsave("images/C1-%s-%d/%d_real.png" % (epoch,batch_i, i), rgb_real)
            #imsave("images/C1-%s-%d/%d_gen.png" % (epoch,batch_i, i), rgb_gen)
            
            fig.savefig("images/C1-%s-%d/turing%d.png" % (epoch,batch_i, i))
            plt.close()
            
        if(save_model):    
            self.save(epoch)
        
    def save(self,ep):
        os.makedirs('saved_model/C1-%s' % ep, exist_ok=True)
        self.combined.save_weights("./saved_model/C1-%s/modelW_epoch_%d.h5" % (ep , ep))
        self.generator.save_weights("./saved_model/C1-%s/genW_epoch_%d.h5" % (ep, ep))
        self.discriminator.save_weights("./saved_model/C1-%s/disW_epoch_%d.h5" % (ep, ep))
        
    def load(self,ep):
        self.combined.load_weights("./saved_model/C1-%s/modelW_epoch_%d.h5" % (ep , ep))
        self.generator.load_weights("./saved_model/C1-%s/genW_epoch_%d.h5" % (ep, ep))
        self.discriminator.load_weights("./saved_model/C1-%s/disW_epoch_%d.h5" % (ep, ep))

In [3]:
gan = C1GAN()
gan.load(10)

In [4]:
gan.train(epochs=10, batch_size=100, sample_interval=175)

C:\Users\miche\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\miche\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/10] [Batch 0/350] [D loss: 0.266566, acc:  32%] [G loss: 1.143466] time: 0:00:16.366317


C:\Users\miche\Anaconda3\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


[Epoch 0/10] [Batch 1/350] [D loss: 0.259885, acc:  53%] [G loss: 1.130720] time: 0:00:29.419494
[Epoch 0/10] [Batch 2/350] [D loss: 0.256177, acc:  57%] [G loss: 1.258416] time: 0:00:35.805417
[Epoch 0/10] [Batch 3/350] [D loss: 0.253612, acc:  50%] [G loss: 1.476810] time: 0:00:42.192338
[Epoch 0/10] [Batch 4/350] [D loss: 0.252900, acc:  48%] [G loss: 1.317581] time: 0:00:48.661043
[Epoch 0/10] [Batch 5/350] [D loss: 0.253167, acc:  46%] [G loss: 1.326573] time: 0:00:55.065917
[Epoch 0/10] [Batch 6/350] [D loss: 0.253152, acc:  50%] [G loss: 1.300932] time: 0:01:01.481731
[Epoch 0/10] [Batch 7/350] [D loss: 0.252400, acc:  49%] [G loss: 1.284121] time: 0:01:07.862670
[Epoch 0/10] [Batch 8/350] [D loss: 0.252998, acc:  43%] [G loss: 1.077934] time: 0:01:14.258569
[Epoch 0/10] [Batch 9/350] [D loss: 0.252083, acc:  50%] [G loss: 1.191026] time: 0:01:20.645523
[Epoch 0/10] [Batch 10/350] [D loss: 0.251629, acc:  47%] [G loss: 1.300182] time: 0:01:27.015459
[Epoch 0/10] [Batch 11/350] [

[Epoch 0/10] [Batch 85/350] [D loss: 0.252129, acc:  45%] [G loss: 1.320943] time: 0:09:21.546659
[Epoch 0/10] [Batch 86/350] [D loss: 0.252062, acc:  49%] [G loss: 1.159334] time: 0:09:27.845816
[Epoch 0/10] [Batch 87/350] [D loss: 0.252746, acc:  51%] [G loss: 1.485478] time: 0:09:34.153950
[Epoch 0/10] [Batch 88/350] [D loss: 0.252264, acc:  50%] [G loss: 1.122989] time: 0:09:40.521923
[Epoch 0/10] [Batch 89/350] [D loss: 0.251572, acc:  50%] [G loss: 1.256689] time: 0:09:46.856019
[Epoch 0/10] [Batch 90/350] [D loss: 0.251632, acc:  50%] [G loss: 1.166589] time: 0:09:53.153182
[Epoch 0/10] [Batch 91/350] [D loss: 0.251602, acc:  49%] [G loss: 1.097389] time: 0:09:59.479235
[Epoch 0/10] [Batch 92/350] [D loss: 0.251778, acc:  46%] [G loss: 1.142729] time: 0:10:05.795382
[Epoch 0/10] [Batch 93/350] [D loss: 0.251510, acc:  48%] [G loss: 1.205928] time: 0:10:12.104478
[Epoch 0/10] [Batch 94/350] [D loss: 0.251521, acc:  49%] [G loss: 1.264405] time: 0:10:18.426609
[Epoch 0/10] [Batch 

...

[Epoch 9/10] [Batch 113/350] [D loss: 0.252368, acc:  50%] [G loss: 1.112292] time: 5:43:20.566942
[Epoch 9/10] [Batch 114/350] [D loss: 0.251485, acc:  47%] [G loss: 1.102094] time: 5:43:26.870120
[Epoch 9/10] [Batch 115/350] [D loss: 0.252055, acc:  48%] [G loss: 1.256508] time: 5:43:33.169246
[Epoch 9/10] [Batch 116/350] [D loss: 0.252145, acc:  57%] [G loss: 1.069794] time: 5:43:39.474388
[Epoch 9/10] [Batch 117/350] [D loss: 0.251763, acc:  53%] [G loss: 1.092171] time: 5:43:45.772547
[Epoch 9/10] [Batch 118/350] [D loss: 0.252555, acc:  49%] [G loss: 1.203545] time: 5:43:52.049794
[Epoch 9/10] [Batch 119/350] [D loss: 0.251501, acc:  44%] [G loss: 1.043971] time: 5:43:58.330004
[Epoch 9/10] [Batch 120/350] [D loss: 0.251326, acc:  56%] [G loss: 1.088828] time: 5:44:04.618158
[Epoch 9/10] [Batch 121/350] [D loss: 0.251451, acc:  50%] [G loss: 1.046381] time: 5:44:10.877422
[Epoch 9/10] [Batch 122/350] [D loss: 0.251190, acc:  45%] [G loss: 1.021435] time: 5:44:17.156664
[Epoch 9/1

C:\Users\miche\Anaconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


[Epoch 9/10] [Batch 176/350] [D loss: 0.252617, acc:  49%] [G loss: 1.121531] time: 5:50:04.370852
[Epoch 9/10] [Batch 177/350] [D loss: 0.251848, acc:  49%] [G loss: 1.190772] time: 5:50:10.935583
[Epoch 9/10] [Batch 178/350] [D loss: 0.250973, acc:  50%] [G loss: 1.252012] time: 5:50:17.546632
[Epoch 9/10] [Batch 179/350] [D loss: 0.251055, acc:  48%] [G loss: 1.062395] time: 5:50:24.041267
[Epoch 9/10] [Batch 180/350] [D loss: 0.251107, acc:  47%] [G loss: 1.045665] time: 5:50:30.354387
[Epoch 9/10] [Batch 181/350] [D loss: 0.251194, acc:  50%] [G loss: 1.120226] time: 5:50:36.955737
[Epoch 9/10] [Batch 182/350] [D loss: 0.251190, acc:  47%] [G loss: 1.012562] time: 5:50:43.404525
[Epoch 9/10] [Batch 183/350] [D loss: 0.250893, acc:  49%] [G loss: 1.190439] time: 5:50:49.729581
[Epoch 9/10] [Batch 184/350] [D loss: 0.250975, acc:  51%] [G loss: 1.186181] time: 5:50:56.039710
[Epoch 9/10] [Batch 185/350] [D loss: 0.251204, acc:  49%] [G loss: 0.932293] time: 5:51:02.354855
[Epoch 9/1

[Epoch 9/10] [Batch 259/350] [D loss: 0.250811, acc:  46%] [G loss: 0.935684] time: 5:58:50.288664
[Epoch 9/10] [Batch 260/350] [D loss: 0.250860, acc:  49%] [G loss: 1.081445] time: 5:58:56.606771
[Epoch 9/10] [Batch 261/350] [D loss: 0.251047, acc:  48%] [G loss: 0.982864] time: 5:59:02.921885
[Epoch 9/10] [Batch 262/350] [D loss: 0.250950, acc:  45%] [G loss: 1.269048] time: 5:59:09.230019
[Epoch 9/10] [Batch 263/350] [D loss: 0.251058, acc:  48%] [G loss: 1.040152] time: 5:59:15.551117
[Epoch 9/10] [Batch 264/350] [D loss: 0.250980, acc:  49%] [G loss: 1.053524] time: 5:59:21.855261
[Epoch 9/10] [Batch 265/350] [D loss: 0.251367, acc:  48%] [G loss: 0.929367] time: 5:59:28.175363
[Epoch 9/10] [Batch 266/350] [D loss: 0.251273, acc:  48%] [G loss: 0.996236] time: 5:59:34.502477
[Epoch 9/10] [Batch 267/350] [D loss: 0.251240, acc:  48%] [G loss: 0.937745] time: 5:59:40.859448
[Epoch 9/10] [Batch 268/350] [D loss: 0.251365, acc:  55%] [G loss: 1.035185] time: 5:59:47.174563
[Epoch 9/1

[Epoch 9/10] [Batch 342/350] [D loss: 0.251307, acc:  46%] [G loss: 1.254246] time: 6:07:36.027147
[Epoch 9/10] [Batch 343/350] [D loss: 0.251288, acc:  49%] [G loss: 1.079780] time: 6:07:42.421084
[Epoch 9/10] [Batch 344/350] [D loss: 0.251662, acc:  45%] [G loss: 1.148721] time: 6:07:48.735200
[Epoch 9/10] [Batch 345/350] [D loss: 0.251563, acc:  51%] [G loss: 1.037053] time: 6:07:55.051312
[Epoch 9/10] [Batch 346/350] [D loss: 0.251328, acc:  47%] [G loss: 1.027043] time: 6:08:01.367393
[Epoch 9/10] [Batch 347/350] [D loss: 0.251465, acc:  55%] [G loss: 1.016329] time: 6:08:07.667548
[Epoch 9/10] [Batch 348/350] [D loss: 0.251553, acc:  49%] [G loss: 0.952170] time: 6:08:13.970727
weights saved...


In [5]:
#gan.save(10)
gan.sample_images(11,0)

C:\Users\miche\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\miche\Anaconda3\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\miche\Anaconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


In [ ]:
from scipy.misc import imsave
data = DataLoader(dataset_name="val_256",
                                      img_res=(256, 256))
imgs_A, imgs_B = data.load_data(batch_size=60, is_testing=True)

r, c = 3, 3

fake_A = gan.generator.predict(imgs_B)

gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5
fake_A = 0.5 * fake_A + 0.5
imgs_B = 0.5 * imgs_B + 0.5
imgs_A = 0.5 * imgs_A + 0.5
titles = ['Originale', 'B&W', 'Generata']


ic = 0
f=0
cnt = 0
for img in fake_A:
    f += 1
    imsave('images/test9000/testOutput%d.png' % (f), img)
    
    if ic==2:
        fig, axs = plt.subplots(r, c)
        axs[0, 0].set_title(titles[0])
        axs[0, 1].set_title(titles[1])
        axs[0, 2].set_title(titles[2])
        for i in range(r):
            axs[i,0].imshow(imgs_A[cnt])
            axs[i,1].imshow(imgs_B[cnt])
            axs[i,2].imshow(fake_A[cnt])
            cnt += 1
        
        fig.savefig("images/test9000/%d.png" % (cnt))
        plt.close()
    if ic == 2:
        ic=0
    else:
        ic += 1
    

In [ ]:
imgs_A, embed ,imgs_B = gan.data_loader.load_data(batch_size=2)
fake_A = gan.generator.predict([imgs_B , embed])
fake_A = fake_A * 128

# Output colorizations
for i in range(len(fake_A)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = imgs_B[i][:,:,0]
    cur[:,:,1:] = fake_A[i]
    imsave("images/test-c1/img_final_"+str(i)+".png", lab2rgb(cur))